In [3]:
import json
import tensorflow as tf
import numpy as np
from typing import Optional
from src.models.ig_cnn import ig_cnn_model


In [4]:
def gen_conv(filters:int, kernel_size:int, activation:Optional[str]=None, padding:Optional[str]=None):
   _conv = f"C-{filters}-{kernel_size}"
   if activation is not None:
       _conv += f"-{activation}"
   if padding is not None:
       _conv += f"-{padding}"
   return _conv

def gen_dense(units: int, activation: Optional[str]=None):
    _dense = f"D-{units}"
    if  activation is not None:
       _dense += f"-{activation}"
    return _dense

### Generate configs

In [5]:
idx = 0
for phantom_size in [3, 5]: # 7]:
    print(f"Input {phantom_size}")
    for activation in ['sigmoid', 'relu']:
        for filters in [10, 15, 25, 50]:
            for kernel in [3, 5, 7]:
                i = phantom_size
                depth = 0
                layers = ""
                while i >= kernel:
                    depth += 1
                    i -= (kernel-np.floor(kernel/2.0))
                    layers += f"{gen_conv(filters, kernel, activation, padding='valid')},"
                    # Test if model is creatable
                    model_def = f"{layers}F"
                    model = ig_cnn_model(model_def, input_size=(phantom_size, phantom_size))
                    model.compile(loss='mse')
                    with open(f"{idx}.json", 'w') as file:
                        config = dict(model_def=model_def,
                                      activation=activation,
                                      kernel=kernel,
                                      filters=filters,
                                      depth=depth,
                                      phantom_size=phantom_size,
                                      log_name=f'{phantom_size}/{model_def}/',
                                      log_root='../logs/',
                                      phantom_root=f'../data/phantoms/phantom_{phantom_size}_{phantom_size}/training',
                                      phantom_file_wildcard=f'*_phantom_{phantom_size}_{phantom_size}.npz'
                                 )
                        json.dump(config, file)
                    idx += 1

Input 3


### Generate configs

In [26]:
idx = 0
for phantom_size in [3]: #, 5, 7]:
    print(f"Input {phantom_size}")
    for activation in ['sigmoid', 'relu']:
        for filters in [10, 15, 25, 50]:
            for kernel in [3, 5, 7]:
                i = phantom_size
                depth = 0
                layers = ""
                while i >= kernel:
                    depth += 1
                    i -= (kernel-np.floor(kernel/2.0))
                    layers += f"{gen_conv(filters, kernel, activation, padding='valid')},"
                    # Test if model is creatable
                    model_def = f"{layers}F"
                    model = ig_cnn_model(model_def, input_size=(phantom_size, phantom_size))
                    model.compile(loss='mse')
                    with open(f"{idx}.json", 'w') as file:
                        config = dict(model_def=model_def,
                                      activation=activation,
                                      kernel=kernel,
                                      filters=filters,
                                      depth=depth,
                                      phantom_size=phantom_size,
                                      log_name=f'{phantom_size}/{model_def}/',
                                      log_root='../logs/',
                                      phantom_root=f'../data/phantoms/phantom_{phantom_size}_{phantom_size}/training',
                                      phantom_file_wildcard=f'*_phantom_{phantom_size}_{phantom_size}.npz'
                                 )
                        json.dump(config, file)

Input 3
